<a href="https://colab.research.google.com/github/pipeton8/6.439-stats-comp-applications/blob/main/Assignments/4%20-%20Networks/6.439%20-%20Problem%20Set%204.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Problem Set 4 - Statistics, Computation and Applications

(c) Felipe del Canto, November 2021
***
***

_Note: The dataset for this assignment was provided for in class use only. For this reason, this notebook will not run. I will try to provide a toy dataset in the future._

# Part 0 (Execute first!)

In [1]:
# Import packages
import scipy
import numpy as np
import pandas as pd
pd.options.display.float_format = '{:.4f}'.format

import seaborn           as sns
import matplotlib.pyplot as plt

from tqdm.auto import tqdm, trange

# Problem 4.2: Co-offending Network

## Preamble

In [21]:
# Load data
co_off = pd.read_csv("Cooffending.csv")

# Set date
co_off['date'] = pd.to_datetime(co_off['date'])

# Set crime codes, counties and locations as categorical
categorical_vars = (['offender_id', 'sex', 'event_id'] 
                    + [f'crime{k}_code' for k in range(1,5)] 
                    + ['county', 'location'])
co_off[categorical_vars] = co_off[categorical_vars].astype('category')

In [22]:
co_off.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1280459 entries, 0 to 1280458
Data columns (total 14 columns):
 #   Column       Non-Null Count    Dtype         
---  ------       --------------    -----         
 0   offender_id  1280459 non-null  category      
 1   dob          1280459 non-null  int64         
 2   sex          1280459 non-null  category      
 3   event_id     1280459 non-null  category      
 4   date         1280459 non-null  datetime64[ns]
 5   year         1280459 non-null  int64         
 6   location     1280459 non-null  category      
 7   county       1280459 non-null  category      
 8   crime1_code  1280459 non-null  category      
 9   crime2_code  1280459 non-null  category      
 10  crime3_code  1280459 non-null  category      
 11  crime4_code  1280459 non-null  category      
 12  minors       1280459 non-null  int64         
 13  adults       1280459 non-null  int64         
dtypes: category(9), datetime64[ns](1), int64(4)
memory usage: 145.1 MB

In [20]:
co_off.describe(include='all', datetime_is_numeric=True)

,offender_id,dob,sex,event_id,date,year,location,county,crime1_code,crime2_code,crime3_code,crime4_code,minors,adults
count,1280459.0000,1280459.0000,1280459.0000,1280459.0000,1280459,1280459.0000,1280459,1280459.0000,1280459,1280459,1280459,1280459,1280459.0000,1280459.0000
unique,539593.0000,nan,2.0000,1164836.0000,NaN,nan,99,1342.0000,295,244,178,116,nan,nan
top,300728.0000,nan,1.0000,27849.0000,NaN,nan,1,66023.0000,1430,,,,nan,nan
freq,456.0000,nan,1079224.0000,156.0000,NaN,nan,190114,330496.0000,138549,1130107,1250407,1273533,nan,nan
mean,nan,1973.4897,nan,nan,2006-11-02 13:31:11.916521472,2006.3336,NaN,nan,NaN,NaN,NaN,NaN,0.0197,1.3399
min,nan,1007.0000,nan,nan,2003-01-01 00:00:00,2003.0000,NaN,nan,NaN,NaN,NaN,NaN,0.0000,0.0000
25%,nan,1964.0000,nan,nan,2005-05-17 00:00:00,2005.0000,NaN,nan,NaN,NaN,NaN,NaN,0.0000,1.0000
50%,nan,1976.0000,nan,nan,2006-12-16 00:00:00,2006.0000,NaN,nan,NaN,NaN,NaN,NaN,0.0000,1.0000
75%,nan,1984.0000,nan,nan,2008-06-18 00:00:00,2008.0000,NaN,nan,NaN,NaN,NaN,NaN,0.0000,1.0000
max,nan,2005.0000,nan,nan,2010-01-25 00:00:00,2010.0000,NaN,nan,NaN,NaN,NaN,NaN,92.0000,156.0000


## (a)

## (b)

## (c)

## (d)

## (e)

## (f)

## (g)

## (h)

## (i)

## (j)

## (k)

## (l)

## (m)